In [1]:
% matplotlib inline

import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
H1_data = np.genfromtxt('H1_times.csv')
L1_data = np.genfromtxt('L1_times.csv')

In [3]:
H1_cbc = H1_data[np.where(H1_data[:,9] == 1)]
H1_burst = H1_data[np.where(H1_data[:,10] == 1)]
H1_stoch = H1_data[np.where(H1_data[:,11] == 1)]

In [6]:
# using multilabels, with i = 0 = cbc,
#                           = 1 = burst,
#                           = 2 = stoch,
#                           = 3 = terrestrial
training = np.array([])
train_labels = np.array([])
for event in H1_data:
    if (event[9] == 1 or event[10] == 1 or event[11] == 1):
        if (len(training) == 0):
            training = np.array([event[2],event[3],event[4],event[5]])
            train_labels = np.array([int(event[9]),int(event[10]),int(event[11]),0])
        else:
            training = np.vstack((training,np.array([event[2],event[3],event[4],event[5]])))
            train_labels = np.vstack((train_labels,np.array([int(event[9]),int(event[10]),int(event[11]),0])))
        
        continue # not counting hardware injections as terrestrial

    
    b = event[0]
    e = event[1]
    
    # find H1 events whose beginings and ends do not overlap with L1 events
    b_novr = len(np.where((b > L1_data[:,0]) & (b < L1_data[:,1]))[0]) == 0
    e_novr = len(np.where((e > L1_data[:,0]) & (e < L1_data[:,1]))[0]) == 0
    
    if (b_novr and e_novr):
        if (len(training) == 0):
            training = np.array([event[2],event[3],event[4],event[5]])
            train_labels = np.array([0,0,0,1])
        else:
            training = np.vstack((training,np.array([event[2],event[3],event[4],event[5]])))
            train_labels = np.vstack((train_labels, np.array([0,0,0,1])))

In [7]:
print(training.shape)
print(train_labels.shape)

(206, 4)
(206, 4)
[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0

In [8]:
# scale data for mlp
scaler = StandardScaler()  

scaler.fit(training)
train_scale = scaler.transform(training)

In [17]:
# MLP params
hidden = (3,)
activ = 'logistic'
solver = 'lbfgs'

In [9]:
mlp = MLPClassifier()
parameters = {'hidden_layer_sizes':[(3,)],'activation':['logistic'],'solver':['lbfgs'],'alpha':(10.0 ** -np.arange(1, 7))}
clf = GridSearchCV(mlp, parameters)
clf.fit(training,train_labels)

GridSearchCV(cv=None, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([  1.00000e-01,   1.00000e-02,   1.00000e-03,   1.00000e-04,
         1.00000e-05,   1.00000e-06]), 'activation': ['logistic'], 'solver': ['lbfgs'], 'hidden_layer_sizes': [(3,)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [18]:
alpha = clf.best_params_['alpha']

mlp = MLPClassifier(hidden_layer_sizes=hidden,activation=activ,solver=solver,alpha=alpha)
mlp.fit(training,train_labels)

MLPClassifier(activation='logistic', alpha=0.10000000000000001,
       batch_size='auto', beta_1=0.9, beta_2=0.999, early_stopping=False,
       epsilon=1e-08, hidden_layer_sizes=(3,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [20]:
mlp.score(training,train_labels)

0.98058252427184467

'log_loss'